In [1]:
import os
os.chdir('example_inputs/')
sc_fermi_exec = '/Users/bjm42/source/sc-fermi/upload-sourceforge/frozen-sc-fermi'
print(os.popen(sc_fermi_exec).read())
os.chdir('..')

**************************************************************
  (FROZEN)

   SSSS    CCCC      FFFFFF  EEEEEE   RRRR   MM     MM  IIIII
  SS   S  CC   C     FF      EE      RR   R  MMM   MMM    I
  SS      CC         FF      EE      RR  R   M MM MM M    I
   SSSS   CC     --- FFFFFF  EEEEEE  RRRR    M  MMM  M    I
      SS  CC         FF      EE      R   R   M   M   M    I
  S   SS  CC   C     FF      EE      R   RR  M   M   M    I
   SSSS    CCCC      FF      EEEEEE  R   RR  M   M   M  IIIII


Energies in eV, temperature in Kelvin, DOS in states/unit cell

------
j.buckeridge@ucl.ac.uk 2018
**************************************************************

unitcell.dat found...
(Should be cell for which DOS was determined!)
 
Volume of cell:      57.788884 A^3
 
Found non-spin polarised system...
 
Number of electrons in system:      18.000000
         Energy gap of system:       0.808400 eV
                 Temperature :     300.000000 K
     Number of defect species:     2
 
Defects f

In [2]:
# py-sc-fermi

import timeit

start = timeit.default_timer()

from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.defect_charge_state import FrozenDefectChargeState
from py_sc_fermi.inputs import inputs_from_files

import numpy as np

In [3]:
example_dir = 'example_inputs'

unitcell_filename = f'{example_dir}/unitcell.dat'
totdos_filename = f'{example_dir}/totdos.dat'
input_fermi_filename = f'{example_dir}/input-fermi.dat'

# TODO: Update the inputs.read_input_data() function to handle frozen-sc-fermi input files.
inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)

Volume of cell: 57.78888409009906 A^3
Found non-spin polarised system...
Number of electrons in system: 18
Energy gap of system: 0.8084 eV
Temperature: 100.0 K
Number of defect species: 2
Reading non-spin-polarised DOS


In [4]:
defect_system = DefectSystem( defect_species=inputs['defect_species'],
                              volume=inputs['volume'],
                              dos=inputs['dos'], # py_sc_fermi.DOS object,
                              temperature=300.0 )

defect_system

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

In [5]:
from collections import Counter
nfrozen = 1
namef = ['V_Ga']
concf = [0.3285677364522e20]
# check frozen defects are included in the original list of defects
for name in namef:
    if name not in defect_system.defect_species_names:
        raise ValueError((f'ERROR: fixed concentration defect "{name}"'+
                           'is not in list of original defects!\n'+
                           'Please add new frozen defects as fixed charge defects in'
                           'a separate list - otherwise a solution cannot be found!!'))
# Check if any defects have been assigned multiple fixed concentrations
c = Counter(namef)
for n, count in c.items():
    if count > 1:
        raise ValueError(f'ERROR: defect "{n}"has multiple fixed concentrations!!')
for n, c in zip(namef, concf):
    defect_system.defect_species_by_name(n).fix_concentration( c * defect_system.volume / 1e24 )
    
defect_system

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
fixed [c] = 0.0018987562837582401 cm^-3
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

In [6]:
ficharg = False
fixconch = False
namefn = ['V_Ga', 'Ga_i']
chargefn = [-1.0, 1.0]
concfn = [0.19e19, 0.5e20]
if 0 in chargefn:
    checkfixchg = True
else:
    checkfixchg = False
for n,q,c  in zip( namefn, chargefn, concfn ):
    print(c)
    c = c * defect_system.volume / 1e24
    frozen_charge_state = FrozenDefectChargeState(charge=q, concentration=c)
    if n in defect_system.defect_species_names:     
        defect_system.defect_species_by_name(n).charge_states[q] = frozen_charge_state
    else:
        ds = DefectSpecies(name=n, nsites=1, charge_states=[frozen_charge_state])
        defect_system.defect_species.append(ds)

defect_system.check_concentrations()
defect_system.report()

1.9e+18
5e+19
SC Fermi level :      0.018315362596523502  (eV)

Concentrations:
n (electrons)  : 609633.7634268429 cm^-3
p (holes)      : 1.3479102448055966e+19 cm^-3
V_Ga           : 3.285677364522e+19 cm^-3
Ga_Sb          : 2.3737107494384077e-13 cm^-3
Ga_i           : 5e+19 cm^-3

Breakdown of concentrations for each defect charge state:
---------------------------------------------------------
V_Ga       : Charge Concentration(cm^-3) Total
           :  0.0  1.672432e+17          0.51 
           : -1.0  1.900000e+18          5.78  [fixed]
           : -2.0  3.078949e+19          93.71 
           : -3.0  4.159159e+13          0.00 
---------------------------------------------------------
Ga_Sb      : Charge Concentration(cm^-3) Total
           :  0.0  1.547224e-16          0.07 
           : -1.0  2.361934e-13          99.50 
           : -2.0  1.022993e-15          0.43 
---------------------------------------------------------
Ga_i       : Charge Concentration(cm^-3) Total
   

/Users/bjm42/.pyenv/versions/3.7.0/lib/python3.7/site-packages/scipy/optimize/_minimize.py:763: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:58: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((edos - e_fermi)/kT))
/Users/bjm42/source/py-sc-fermi/py_sc_fermi/dos.py:55: RuntimeWarning: overflow encountered in exp
  return dos / (1.0 + np.exp((e_fermi - edos)/kT))
